In [62]:
import MetaTrader5 as mt5
from datetime import datetime, timedelta, time
from IPython.display import display
import plotly.express as px

from projects.atj_trading_legacy.backtester import Backtester, get_ohlc_history, create_price_fig, evaluate_backtest

In [63]:
symbol = 'EURUSD'
start_dt = datetime(2020, 1, 1)
end_dt = datetime.now()

# get historical data
mt5.initialize()
ohlc = get_ohlc_history(symbol, mt5.TIMEFRAME_D1, start_dt, end_dt, additional_columns=['tick_volume'])
ohlc['date'] = ohlc['time'].dt.date
display(ohlc)

fig = create_price_fig(ohlc)
display(fig)

,time,open,high,low,close,tick_volume,date
0,2020-01-02,1.12119,1.12246,1.11636,1.11726,40413,2020-01-02
1,2020-01-03,1.11717,1.11798,1.11250,1.11568,48272,2020-01-03
2,2020-01-06,1.11577,1.12056,1.11572,1.11969,41393,2020-01-06
3,2020-01-07,1.11968,1.11974,1.11336,1.11517,44674,2020-01-07
4,2020-01-08,1.11519,1.11682,1.11018,1.11041,64999,2020-01-08
...,...,...,...,...,...,...,...
1203,2024-08-20,1.10795,1.11305,1.10719,1.11291,52419,2024-08-20
1204,2024-08-21,1.11231,1.11742,1.10988,1.11498,61075,2024-08-21
1205,2024-08-22,1.11470,1.11646,1.10982,1.11111,54436,2024-08-22
1206,2024-08-23,1.11045,1.12009,1.11045,1.11924,61408,2024-08-23


In [64]:
# wvap = (cumulative typical price X volume) / Cumulative Volume
# Typical price = (High + Low + closing) /3
# Cumulative Volume starts with the start of the day

# ToDO: Bollinger Bands using VWAP STD
# ToDo: No Rolling, Start Calculation at the beginning of each day

In [65]:
def calc_typical_price(x):
    return (x['high'] + x['low'] + x['close']) / 3

ohlc['typical_price'] = ohlc.apply(calc_typical_price, axis=1)
ohlc

,time,open,high,low,close,tick_volume,date,typical_price
0,2020-01-02,1.12119,1.12246,1.11636,1.11726,40413,2020-01-02,1.118693
1,2020-01-03,1.11717,1.11798,1.11250,1.11568,48272,2020-01-03,1.115387
2,2020-01-06,1.11577,1.12056,1.11572,1.11969,41393,2020-01-06,1.118657
3,2020-01-07,1.11968,1.11974,1.11336,1.11517,44674,2020-01-07,1.116090
4,2020-01-08,1.11519,1.11682,1.11018,1.11041,64999,2020-01-08,1.112470
...,...,...,...,...,...,...,...,...
1203,2024-08-20,1.10795,1.11305,1.10719,1.11291,52419,2024-08-20,1.111050
1204,2024-08-21,1.11231,1.11742,1.10988,1.11498,61075,2024-08-21,1.114093
1205,2024-08-22,1.11470,1.11646,1.10982,1.11111,54436,2024-08-22,1.112463
1206,2024-08-23,1.11045,1.12009,1.11045,1.11924,61408,2024-08-23,1.116593


In [66]:
ohlc['cumulative_volume'] = ohlc.groupby('date')['tick_volume'].cumsum()
ohlc['price_x_volume'] = ohlc['typical_price'] * ohlc['tick_volume']
ohlc['cumulative_pxv'] =  ohlc.groupby('date')['price_x_volume'].cumsum()
ohlc['vwap'] = ohlc['cumulative_pxv'] / ohlc['cumulative_volume']

ohlc['vwap'] = ohlc['vwap'].shift(1)

fig = px.line(ohlc, x='time', y=['vwap', 'close'])
display(fig)

display(ohlc)

,time,open,high,low,close,tick_volume,date,typical_price,cumulative_volume,price_x_volume,cumulative_pxv,vwap
0,2020-01-02,1.12119,1.12246,1.11636,1.11726,40413,2020-01-02,1.118693,40413,45209.753680,45209.753680,NaN
1,2020-01-03,1.11717,1.11798,1.11250,1.11568,48272,2020-01-03,1.115387,48272,53841.945173,53841.945173,1.118693
2,2020-01-06,1.11577,1.12056,1.11572,1.11969,41393,2020-01-06,1.118657,41393,46304.555403,46304.555403,1.115387
3,2020-01-07,1.11968,1.11974,1.11336,1.11517,44674,2020-01-07,1.116090,44674,49860.204660,49860.204660,1.118657
4,2020-01-08,1.11519,1.11682,1.11018,1.11041,64999,2020-01-08,1.112470,64999,72309.437530,72309.437530,1.116090
...,...,...,...,...,...,...,...,...,...,...,...,...
1203,2024-08-20,1.10795,1.11305,1.10719,1.11291,52419,2024-08-20,1.111050,52419,58240.129950,58240.129950,1.106447
1204,2024-08-21,1.11231,1.11742,1.10988,1.11498,61075,2024-08-21,1.114093,61075,68043.250333,68043.250333,1.111050
1205,2024-08-22,1.11470,1.11646,1.10982,1.11111,54436,2024-08-22,1.112463,54436,60558.054013,60558.054013,1.114093
1206,2024-08-23,1.11045,1.12009,1.11045,1.11924,61408,2024-08-23,1.116593,61408,68567.763413,68567.763413,1.112463


In [67]:
ohlc['vsma_20'] = ohlc['vwap'].rolling(100).mean()
ohlc['vstd'] = ohlc['vwap'].rolling(100).std()

num_std = 2.5

ohlc['upper_band'] = ohlc['vsma_20'] + num_std * ohlc['vstd']
ohlc['lower_band'] = ohlc['vsma_20'] - num_std * ohlc['vstd']

px.line(ohlc, x='time', y=['upper_band', 'lower_band', 'vwap', 'close'])

In [68]:
def get_signal(x):
    if x['vwap'] > x['upper_band']:
        return 'buy'
    elif x['vwap'] < x['lower_band']:
        return 'sell'
    
ohlc['signal'] = ohlc.apply(get_signal, axis=1)
ohlc

,time,open,high,low,close,tick_volume,date,typical_price,cumulative_volume,price_x_volume,cumulative_pxv,vwap,vsma_20,vstd,upper_band,lower_band,signal
0,2020-01-02,1.12119,1.12246,1.11636,1.11726,40413,2020-01-02,1.118693,40413,45209.753680,45209.753680,NaN,NaN,NaN,NaN,NaN,None
1,2020-01-03,1.11717,1.11798,1.11250,1.11568,48272,2020-01-03,1.115387,48272,53841.945173,53841.945173,1.118693,NaN,NaN,NaN,NaN,None
2,2020-01-06,1.11577,1.12056,1.11572,1.11969,41393,2020-01-06,1.118657,41393,46304.555403,46304.555403,1.115387,NaN,NaN,NaN,NaN,None
3,2020-01-07,1.11968,1.11974,1.11336,1.11517,44674,2020-01-07,1.116090,44674,49860.204660,49860.204660,1.118657,NaN,NaN,NaN,NaN,None
4,2020-01-08,1.11519,1.11682,1.11018,1.11041,64999,2020-01-08,1.112470,64999,72309.437530,72309.437530,1.116090,NaN,NaN,NaN,NaN,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1203,2024-08-20,1.10795,1.11305,1.10719,1.11291,52419,2024-08-20,1.111050,52419,58240.129950,58240.129950,1.106447,1.080744,0.009176,1.103683,1.057805,buy
1204,2024-08-21,1.11231,1.11742,1.10988,1.11498,61075,2024-08-21,1.114093,61075,68043.250333,68043.250333,1.111050,1.081097,0.009649,1.105218,1.056976,buy
1205,2024-08-22,1.11470,1.11646,1.10982,1.11111,54436,2024-08-22,1.112463,54436,60558.054013,60558.054013,1.114093,1.081425,0.010197,1.106918,1.055933,buy
1206,2024-08-23,1.11045,1.12009,1.11045,1.11924,61408,2024-08-23,1.116593,61408,68567.763413,68567.763413,1.112463,1.081703,0.010655,1.108340,1.055066,buy


In [69]:
# create trade logic
def on_bar(data, trades, orders):
    volume = 100000 # 1 lots
    
    open_trades = trades[trades['state'] == 'open']
    num_open_trades = open_trades.shape[0]
    
    # entry signal
    if data['signal'] == 'buy' and not num_open_trades:
        orders.open_trade(symbol, volume, 'buy', tp=data['open'] + 2 * data['vstd'], sl=data['open'] - 2 * data['vstd'])
    
    elif data['signal'] == 'sell' and not num_open_trades:
        orders.open_trade(symbol, volume, 'sell', tp=data['open'] - 2 * data['vstd'], sl=data['open'] + 2 * data['vstd'])

"""
    # exit signal
    if num_open_trades:
        trade = open_trades.iloc[0]

        if trade['order_type'] == 'buy' and data['signal'] == 'sell':
            orders.close_trade(trade)
        elif trade['order_type'] == 'sell' and data['signal'] == 'buy':
            orders.close_trade(trade)
"""

"\n    # exit signal\n    if num_open_trades:\n        trade = open_trades.iloc[0]\n\n        if trade['order_type'] == 'buy' and data['signal'] == 'sell':\n            orders.close_trade(trade)\n        elif trade['order_type'] == 'sell' and data['signal'] == 'buy':\n            orders.close_trade(trade)\n"

In [70]:
# backtest parameters
starting_balance = 10000
currency = 'USD'
exchange_rate = 1
commission = -7 / 100000

# backtest
bt = Backtester()
bt.set_starting_balance(starting_balance, currency=currency)
bt.set_exchange_rate(exchange_rate)
bt.set_commission(commission)

bt.set_historical_data(ohlc)
bt.set_on_bar(on_bar)

bt.run_backtest()

bt.trades

,state,symbol,order_type,volume,open_time,open_price,close_time,close_price,sl,tp,info,profit,commission,profit_net,profit_cumulative,balance
0,closed,EURUSD,buy,100000,2020-06-11 00:00:00,1.13729,2020-07-27 00:00:00,1.169242,1.105338,1.169242,{},3195.24,-7.0,3188.24,3188.24,13188.24
1,closed,EURUSD,buy,100000,2020-07-28 00:00:00,1.17513,2020-12-17 00:00:00,1.222466,1.127794,1.222466,{},4733.59,-7.0,4726.59,7914.83,17914.83
2,closed,EURUSD,buy,100000,2020-12-18 00:00:00,1.22673,2021-02-03 00:00:00,1.200846,1.200846,1.252614,{},-2588.39,-7.0,-2595.39,5319.44,15319.44
3,closed,EURUSD,sell,100000,2021-11-16 00:00:00,1.1367,2022-01-28 00:00:00,1.113199,1.160201,1.113199,{},2350.09,-7.0,2343.09,7662.53,17662.53
4,closed,EURUSD,sell,100000,2022-03-07 00:00:00,1.09266,2022-04-26 00:00:00,1.06518,1.12014,1.06518,{},2748.00,-7.0,2741.0,10403.53,20403.53
5,closed,EURUSD,sell,100000,2022-04-28 00:00:00,1.05603,2022-07-08 00:00:00,1.013349,1.098711,1.013349,{},4268.14,-7.0,4261.14,14664.67,24664.67
6,closed,EURUSD,sell,100000,2022-07-13 00:00:00,1.00372,2022-12-05 00:00:00,1.058074,1.058074,0.949366,{},-5435.43,-7.0,-5442.43,9222.24,19222.24
7,closed,EURUSD,buy,100000,2023-07-17 00:00:00,1.12308,2023-07-28 00:00:00,1.094542,1.094542,1.151618,{},-2853.81,-7.0,-2860.81,6361.43,16361.43
8,closed,EURUSD,sell,100000,2024-04-16 00:00:00,1.06216,2024-05-03 00:00:00,1.079311,1.079311,1.045009,{},-1715.13,-7.0,-1722.13,4639.3,14639.3
9,closed,EURUSD,buy,100000,2024-08-15 00:00:00,1.10105,2024-08-23 00:00:00,1.117814,1.084286,1.117814,{},1676.42,-7.0,1669.42,6308.72,16308.72


In [74]:
pnl_chart = bt.plot_pnl()
pnl_chart

In [75]:
backtest_fig = bt.visualize_backtest(indicators=['vwap', 'upper_band', 'lower_band'])
backtest_fig

In [76]:
bt.export_to_json('vwap_backtest.json', indicators=['upper_band', 'lower_band'])

1

In [73]:
evaluate_backtest(bt.trades)

biggest_profit 4733.59
biggest_loss -5435.43


,state,symbol,order_type,volume,open_time,open_price,close_time,close_price,sl,tp,info,profit,commission,profit_net,profit_cumulative,balance
0,closed,EURUSD,buy,100000,2020-06-11,1.13729,2020-07-27,1.169242,1.105338,1.169242,{},3195.24,-7.0,3188.24,3188.24,13188.24
1,closed,EURUSD,buy,100000,2020-07-28,1.17513,2020-12-17,1.222466,1.127794,1.222466,{},4733.59,-7.0,4726.59,7914.83,17914.83
3,closed,EURUSD,sell,100000,2021-11-16,1.1367,2022-01-28,1.113199,1.160201,1.113199,{},2350.09,-7.0,2343.09,7662.53,17662.53
4,closed,EURUSD,sell,100000,2022-03-07,1.09266,2022-04-26,1.06518,1.12014,1.06518,{},2748.00,-7.0,2741.0,10403.53,20403.53
5,closed,EURUSD,sell,100000,2022-04-28,1.05603,2022-07-08,1.013349,1.098711,1.013349,{},4268.14,-7.0,4261.14,14664.67,24664.67
9,closed,EURUSD,buy,100000,2024-08-15,1.10105,2024-08-23,1.117814,1.084286,1.117814,{},1676.42,-7.0,1669.42,6308.72,16308.72


,state,symbol,order_type,volume,open_time,open_price,close_time,close_price,sl,tp,info,profit,commission,profit_net,profit_cumulative,balance
2,closed,EURUSD,buy,100000,2020-12-18,1.22673,2021-02-03,1.200846,1.200846,1.252614,{},-2588.39,-7.0,-2595.39,5319.44,15319.44
6,closed,EURUSD,sell,100000,2022-07-13,1.00372,2022-12-05,1.058074,1.058074,0.949366,{},-5435.43,-7.0,-5442.43,9222.24,19222.24
7,closed,EURUSD,buy,100000,2023-07-17,1.12308,2023-07-28,1.094542,1.094542,1.151618,{},-2853.81,-7.0,-2860.81,6361.43,16361.43
8,closed,EURUSD,sell,100000,2024-04-16,1.06216,2024-05-03,1.079311,1.079311,1.045009,{},-1715.13,-7.0,-1722.13,4639.3,14639.3
10,closed,EURUSD,buy,100000,2024-08-26,1.11853,2024-08-26,1.11639,1.096107,1.140953,{},-214.00,-7.0,-221.0,6087.72,16087.72


avg_win 3161.913333333334
avg_loss -2561.352
count_profit_trades 6
count_loss_trades 5
win_rate 1.2
rrr 1.23


,order_type,profit
0,buy,3949.05
1,sell,2215.67


,state,symbol,order_type,volume,open_time,open_price,close_time,close_price,sl,tp,info,profit,commission,profit_net,profit_cumulative,balance,dayofweek
0,closed,EURUSD,buy,100000,2020-06-11,1.13729,2020-07-27,1.169242,1.105338,1.169242,{},3195.24,-7.0,3188.24,3188.24,13188.24,0
1,closed,EURUSD,buy,100000,2020-07-28,1.17513,2020-12-17,1.222466,1.127794,1.222466,{},4733.59,-7.0,4726.59,7914.83,17914.83,3
2,closed,EURUSD,buy,100000,2020-12-18,1.22673,2021-02-03,1.200846,1.200846,1.252614,{},-2588.39,-7.0,-2595.39,5319.44,15319.44,2
3,closed,EURUSD,sell,100000,2021-11-16,1.1367,2022-01-28,1.113199,1.160201,1.113199,{},2350.09,-7.0,2343.09,7662.53,17662.53,4
4,closed,EURUSD,sell,100000,2022-03-07,1.09266,2022-04-26,1.06518,1.12014,1.06518,{},2748.00,-7.0,2741.0,10403.53,20403.53,1
5,closed,EURUSD,sell,100000,2022-04-28,1.05603,2022-07-08,1.013349,1.098711,1.013349,{},4268.14,-7.0,4261.14,14664.67,24664.67,4
6,closed,EURUSD,sell,100000,2022-07-13,1.00372,2022-12-05,1.058074,1.058074,0.949366,{},-5435.43,-7.0,-5442.43,9222.24,19222.24,0
7,closed,EURUSD,buy,100000,2023-07-17,1.12308,2023-07-28,1.094542,1.094542,1.151618,{},-2853.81,-7.0,-2860.81,6361.43,16361.43,4
8,closed,EURUSD,sell,100000,2024-04-16,1.06216,2024-05-03,1.079311,1.079311,1.045009,{},-1715.13,-7.0,-1722.13,4639.3,14639.3,4
9,closed,EURUSD,buy,100000,2024-08-15,1.10105,2024-08-23,1.117814,1.084286,1.117814,{},1676.42,-7.0,1669.42,6308.72,16308.72,4


,state,symbol,order_type,volume,open_time,open_price,close_time,close_price,sl,tp,info,profit,commission,profit_net,profit_cumulative,balance,dayofweek,current_max,drawdown
0,closed,EURUSD,buy,100000,2020-06-11,1.13729,2020-07-27,1.169242,1.105338,1.169242,{},3195.24,-7.0,3188.24,3188.24,13188.24,0,3188.24,0.0
1,closed,EURUSD,buy,100000,2020-07-28,1.17513,2020-12-17,1.222466,1.127794,1.222466,{},4733.59,-7.0,4726.59,7914.83,17914.83,3,7914.83,0.0
2,closed,EURUSD,buy,100000,2020-12-18,1.22673,2021-02-03,1.200846,1.200846,1.252614,{},-2588.39,-7.0,-2595.39,5319.44,15319.44,2,7914.83,-2595.39
3,closed,EURUSD,sell,100000,2021-11-16,1.1367,2022-01-28,1.113199,1.160201,1.113199,{},2350.09,-7.0,2343.09,7662.53,17662.53,4,7914.83,-252.3
4,closed,EURUSD,sell,100000,2022-03-07,1.09266,2022-04-26,1.06518,1.12014,1.06518,{},2748.00,-7.0,2741.0,10403.53,20403.53,1,10403.53,0.0
5,closed,EURUSD,sell,100000,2022-04-28,1.05603,2022-07-08,1.013349,1.098711,1.013349,{},4268.14,-7.0,4261.14,14664.67,24664.67,4,14664.67,0.0
6,closed,EURUSD,sell,100000,2022-07-13,1.00372,2022-12-05,1.058074,1.058074,0.949366,{},-5435.43,-7.0,-5442.43,9222.24,19222.24,0,14664.67,-5442.43
7,closed,EURUSD,buy,100000,2023-07-17,1.12308,2023-07-28,1.094542,1.094542,1.151618,{},-2853.81,-7.0,-2860.81,6361.43,16361.43,4,14664.67,-8303.24
8,closed,EURUSD,sell,100000,2024-04-16,1.06216,2024-05-03,1.079311,1.079311,1.045009,{},-1715.13,-7.0,-1722.13,4639.3,14639.3,4,14664.67,-10025.37
9,closed,EURUSD,buy,100000,2024-08-15,1.10105,2024-08-23,1.117814,1.084286,1.117814,{},1676.42,-7.0,1669.42,6308.72,16308.72,4,14664.67,-8355.95


ValueError: Plotly Express cannot process wide-form data with columns of different type.